# Chapter 19 – Training and Deploying TensorFlow Models at Scale

This notebook contains all the sample code and solutions to the exercises in chapter 19.

## Setup
This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

**Warning**: the latest TensorFlow versions are based on Keras 3. For chapters 10-15, it wasn't too hard to update the code to support Keras 3, but unfortunately it's much harder for this chapter, so I've had to revert to Keras 2. To do that, I set the ```TF_USE_LEGACY_KERAS ```environment variable to ```"1" ```and import the ```tf_keras``` package. This ensures that ```tf.keras``` points to ```tf_keras```, which is Keras 2.*.